In [ ]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams["savefig.dpi"] = 300
plt.rcParams["savefig.bbox"] = "tight"

np.set_printoptions(precision=3)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler

# Gradient Boosting

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, stratify=iris.target, random_state=0)
gbrt = GradientBoostingClassifier().fit(X_train, y_train)
gbrt.score(X_test, y_test)

# Illustration on synthetic regression dataset

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

def make_wave(n_samples=100):
    rnd = np.random.RandomState(42)
    x = rnd.uniform(-3, 3, size=n_samples)
    y_no_noise = (np.sin(4 * x))
    y = (y_no_noise + rnd.normal(scale=0.2, size=len(x))) / 2
    return x.reshape(-1, 1), y
X, y = make_wave(100)

def make_poly(n_samples=100):
    rnd = np.random.RandomState(42)
    x = rnd.uniform(-3, 3, size=n_samples)
    y_no_noise = (x) ** 3
    y = (y_no_noise + rnd.normal(scale=3, size=len(x))) / 2
    return x.reshape(-1, 1), y

X, y = make_poly()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=10, learning_rate=.3, random_state=0).fit(X_train, y_train)
gbrt.score(X_test, y_test)

In [ ]:
line = np.linspace(X.min(), X.max(), 1000)
preds = list(gbrt.staged_predict(line[:, np.newaxis]))
plt.plot(X_train[:, 0], y_train, 'o', label="training data")
for i in range(1, 10, 2):
    plt.plot(line, preds[i], label="n_estimators={}".format(i + 1))

plt.legend()

In [ ]:
preds_train = [np.zeros(len(y_train))] + list(gbrt.staged_predict(X_train))


fig, axes = plt.subplots(4, 2, subplot_kw={'xticks': (()), 'yticks': (())}, figsize=(6, 6))
#fig, axes = plt.subplots(4, 2, figsize=(12, 12))

n_estimators = [0, 1, 4, 8]
for i in range(4):
    k = n_estimators[i]
    axes[i, 0].plot(X_train[:, 0], y_train - preds_train[k], 'o')
    axes[i, 0].plot(line, gbrt.estimators_[k, 0].predict(line[:, np.newaxis]))
    axes[i, 1].plot(X_train[:, 0], y_train, 'o')
    axes[i, 1].plot(line, preds[k])
    axes[i, 0].set_title("Residual prediction step {}".format(k + 1))
    axes[i, 1].set_title("Total prediction step {}".format(k + 1))

for ax in axes.ravel():
    ax.set_ylim(y.min(), y.max())
plt.tight_layout()
plt.savefig("images/grad_boost_regression_steps.png")

# illustration on synthetic moon data


In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(noise=.2, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)
gbrt = GradientBoostingClassifier(max_depth=2, n_estimators=10, learning_rate=.2, random_state=0).fit(X_train, y_train)
gbrt.score(X_test, y_test)

In [ ]:
# create a grid for plotting decision functions...
x_lin = np.linspace(X_train[:, 0].min() - .5, X_train[:, 0].max() + .5, 100)
y_lin = np.linspace(X_train[:, 1].min() - .5, X_train[:, 1].max() + .5, 100)
x_grid, y_grid = np.meshgrid(x_lin, y_lin)
X_grid = np.c_[x_grid.ravel(), y_grid.ravel()]

probs = list(gbrt.staged_predict_proba(X_grid))


In [ ]:
fig, axes = plt.subplots(2, 4, subplot_kw={'xticks': (()), 'yticks': (())}, figsize=(12, 6))
for i, (prob, ax) in enumerate(zip(probs, axes.ravel())):
    ax.set_title("n_estimators={}".format(i + 1))
    ax.contourf(x_grid, y_grid, prob[:, 1].reshape(x_grid.shape), alpha=.4, cmap='bwr')
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='bwr', s=10)
fig.suptitle("GradientBostingClassifier(max_depth=2)")
plt.savefig("images/grad_boost_depth2.png")

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'learning_rate': [.2, .1, .05, .02, .01, .001]}
grid = GridSearchCV(GradientBoostingClassifier(),
                    param_grid=param_grid, cv=10,
                    return_train_score=True)
grid.fit(X_train, y_train)

In [ ]:
scores = pd.DataFrame(grid.cv_results_)
scores.plot(x='param_learning_rate', y='mean_train_score', yerr='std_train_score', ax=plt.gca())
scores.plot(x='param_learning_rate', y='mean_test_score', yerr='std_test_score', ax=plt.gca())

In [ ]:
from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingRegressor
boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(
    boston.data, boston.target, random_state=0)

gbrt = GradientBoostingRegressor().fit(X_train, y_train)
gbrt.score(X_test, y_test)

In [ ]:
param_grid = {'learning_rate': [.5, .2, .1, .05, .02, .01, .001]}
grid = GridSearchCV(GradientBoostingRegressor(), param_grid=param_grid, cv=10,
                   return_train_score=True)
grid.fit(X_train, y_train);

In [ ]:
scores = pd.DataFrame(grid.cv_results_)

scores.plot(x='param_learning_rate', y='mean_train_score', yerr='std_train_score', ax=plt.gca())
scores.plot(x='param_learning_rate', y='mean_test_score', yerr='std_test_score', ax=plt.gca())

In [ ]:
grid.best_params_, grid.best_score_

In [ ]:
gbrt.feature_importances_

In [ ]:
np.argsort(gbrt.feature_importances_)[-5:]

In [ ]:
boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(
    boston.data, boston.target, random_state=0)

gbrt = GradientBoostingRegressor().fit(X_train, y_train)
gbrt.score(X_test, y_test)

In [ ]:
from sklearn.ensemble.partial_dependence import plot_partial_dependence
fig, axs = plot_partial_dependence(gbrt, X_train, np.argsort(gbrt.feature_importances_)[-6:],
                                       feature_names=boston.feature_names,
                                       n_jobs=3, grid_resolution=50)
plt.tight_layout()

In [ ]:
fig, axs = plot_partial_dependence(gbrt, X_train, [np.argsort(gbrt.feature_importances_)[-2:]],
                                   feature_names=boston.feature_names,

                                   n_jobs=3, grid_resolution=50)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, stratify=iris.target, random_state=0)
gbrt = GradientBoostingClassifier().fit(X_train, y_train)
gbrt.score(X_test, y_test)

from sklearn.ensemble.partial_dependence import plot_partial_dependence
for i in range(3):
    fig, axs = plot_partial_dependence(gbrt, X_train, range(4), n_cols=4,
                                       feature_names=iris.feature_names, grid_resolution=50, label=i,
                                       figsize=(8, 2))
    fig.suptitle(iris.target_names[i])
    for ax in axs: ax.set_xticks(())
    for ax in axs[1:]: ax.set_ylabel("")

    plt.tight_layout()


# XGBoost

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
print("accuracy on training set: %f" % xgb.score(X_train, y_train))
print("accuracy on test set: %f" % xgb.score(X_test, y_test))

In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle

mnist = fetch_mldata("MNIST original")
X_train, y_train = mnist.data[:60000], mnist.target[:60000]
X_test, y_test = mnist.data[60000:], mnist.target[60000:]

X_train, y_train = shuffle(X_train, y_train)

In [ ]:
# this is not a real benchmark, just illustration!
from time import time
times_xgb = []
acc_xgb = []
times_sklearn = []
acc_sklearn = []
n_samples_ = [10, 100, 1000, 10000]
for n_samples in n_samples_:
    print(n_samples)
    # both do 100 trees, 0.1 learning rate by default
    xgb = XGBClassifier()
    tick = time()
    xgb.fit(X_train[:n_samples], y_train[:n_samples])
    times_xgb.append(time() - tick)
    acc_xgb.append(xgb.score(X_test, y_test))
    
    gbrt = GradientBoostingClassifier()
    tick = time()
    gbrt.fit(X_train[:n_samples], y_train[:n_samples])
    times_sklearn.append(time() - tick)
    acc_sklearn.append(gbrt.score(X_test, y_test))

In [ ]:
from time import time
times_xgb_hist = []
acc_xgb_hist = []
n_samples_ = [10, 100, 1000, 10000]
for n_samples in n_samples_:
    print(n_samples)
    # both do 100 trees, 0.1 learning rate by default
    xgb = XGBClassifier(tree_method="hist")
    tick = time()
    xgb.fit(X_train[:n_samples], y_train[:n_samples])
    times_xgb_hist.append(time() - tick)
    acc_xgb_hist.append(xgb.score(X_test, y_test))

In [ ]:
tx, = plt.plot(n_samples_, times_xgb, label="time xgb")
#tx_hist, = plt.plot(n_samples_, times_xgb_hist, label="time xgb hist")
tsk, = plt.plot(n_samples_, times_sklearn, label="time sklearn")
plt.ylabel("training time")
plt.twinx()
ax, = plt.plot(n_samples_, acc_xgb, '--', label="acc xgb")
#ax_hist, = plt.plot(n_samples_, acc_xgb_hist, '--', label="acc xgb hist")
ask, = plt.plot(n_samples_, acc_sklearn, '--', label="acc sklearn")
plt.ylabel("test accuracy")

plt.xscale("log")
plt.legend((tx, tsk, ax, ask), ('time xgb', 'time sklearn', 'acc xgb', 'acc sklearn'))
plt.savefig("images/xgboost_sklearn_bench.png")

In [ ]:
tx, = plt.plot(n_samples_, times_xgb, label="time xgb")
tx_hist, = plt.plot(n_samples_, times_xgb_hist, label="time xgb hist")
#tsk, = plt.plot(n_samples_, times_sklearn, label="time sklearn")
plt.ylabel("training time")
plt.twinx()
ax, = plt.plot(n_samples_, acc_xgb, '--', label="acc xgb")
ax_hist, = plt.plot(n_samples_, acc_xgb_hist, '--', label="acc xgb hist")
#ask, = plt.plot(n_samples_, acc_sklearn, '--', label="acc sklearn")
plt.ylabel("test accuracy")

plt.xscale("log")
plt.legend((tx, tx_hist, ax, ax_hist), ('time xgb', 'time xgb hist', 'acc xgb', 'acc xgb hist'))
plt.savefig("images/xgboost_hist_bench.png")

# poor man's stacking

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

X, y = make_moons(noise=.4, random_state=192, n_samples=300)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

voting = VotingClassifier([('logreg', LogisticRegression(C=100)),
                           ('tree', DecisionTreeClassifier(max_depth=3, random_state=0)),
                           ('knn', KNeighborsClassifier(n_neighbors=3))
                          ],
                         voting='soft', flatten_transform=True)
voting.fit(X_train, y_train)
lr, tree, knn = voting.estimators_
print(("{:.2f} " * 4).format(voting.score(X_test, y_test),
                             lr.score(X_test, y_test), tree.score(X_test, y_test),
                             knn.score(X_test, y_test)))

In [ ]:
x_lin = np.linspace(X_train[:, 0].min() - .5, X_train[:, 0].max() + .5, 100)
y_lin = np.linspace(X_train[:, 1].min() - .5, X_train[:, 1].max() + .5, 100)
x_grid, y_grid = np.meshgrid(x_lin, y_lin)
X_grid = np.c_[x_grid.ravel(), y_grid.ravel()]
# transform produces individual probabilities
_, lr_probs, _, tree_probs, _, knn_probs = voting.transform(X_grid).T

In [ ]:
fig, axes = plt.subplots(1, 4, subplot_kw={'xticks': (()), 'yticks': (())}, figsize=(8, 2))
titles = ['Logistic Regression', 'Decision Tree', 'KNN', 'Average']
for prob, title, ax in zip([lr_probs, tree_probs, knn_probs, lr_probs + tree_probs + knn_probs], titles, axes.ravel()):
    ax.set_title(title)
    ax.contourf(x_grid, y_grid, prob.reshape(x_grid.shape), alpha=.4, cmap='bwr')
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='bwr', s=10)
plt.savefig("images/average_voting.png")

In [ ]:
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# dropping the negative class probabilities
stacking = make_pipeline(voting, FunctionTransformer(lambda X: X[:, 1::2]), LogisticRegression())
stacking.fit(X_train, y_train)
stacking.score(X_train, y_train)

In [ ]:
stacking.score(X_test, y_test)

In [ ]:
stacking.named_steps.logisticregression.coef_

In [ ]:
ax = plt.gca()
prob = stacking.predict_proba(X_grid)
ax.contourf(x_grid, y_grid, prob[:, 1].reshape(x_grid.shape), alpha=.4, cmap='bwr')
ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='bwr')
ax.set_xticks(())
ax.set_yticks(())
ax.set_title("Logistic Regression weighted")
plt.savefig("images/simple_stacking_result.png")

# Proper stacking

In [ ]:
from sklearn.model_selection import cross_val_predict
first_stage = make_pipeline(voting, FunctionTransformer(lambda X: X[:, 1::2]))
transform_cv = cross_val_predict(first_stage, X_train, y_train, cv=10, method="transform")

In [ ]:
second_stage = LogisticRegression().fit(transform_cv, y_train)
print(second_stage.coef_)

In [ ]:
second_stage.score(transform_cv, y_train)

In [ ]:
second_stage.score(first_stage.transform(X_test), y_test)

In [ ]:
ax = plt.gca()
prob = second_stage.predict_proba(first_stage.transform(X_grid))
ax.contourf(x_grid, y_grid, prob[:, 1].reshape(x_grid.shape), alpha=.4, cmap='bwr')
ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='bwr')

ax.set_xticks(())
ax.set_yticks(())
plt.title("Proper stacking")
plt.savefig("images/stacking_result.png")

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

for i in range(1, 200):
    X, y = make_moons(noise=.4, random_state=i, n_samples=300)
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

    voting = VotingClassifier([('logreg', LogisticRegression(C=100)),
                               ('tree', DecisionTreeClassifier(max_depth=3, random_state=0)),
                               ('knn', KNeighborsClassifier(n_neighbors=3))
                              ],
                             voting='soft', flatten_transform=True)
    voting.fit(X_train, y_train)
    lr, tree, knn = voting.estimators_
    scores = [voting.score(X_test, y_test),
                                 lr.score(X_test, y_test), tree.score(X_test, y_test),
                                 knn.score(X_test, y_test)]

    # dropping the negative class probabilities
    stacking = make_pipeline(voting, FunctionTransformer(lambda X: X[:, 1::2]), LogisticRegression())
    stacking.fit(X_train, y_train)
    naive_train_score = stacking.score(X_train, y_train)

    naive_test_score = stacking.score(X_test, y_test)

    stacking.named_steps.logisticregression.coef_

    # Proper stacking

    from sklearn.model_selection import cross_val_predict
    first_stage = make_pipeline(voting, FunctionTransformer(lambda X: X[:, 1::2]))
    transform_cv = cross_val_predict(first_stage, X_train, y_train, cv=10, method="transform")

    second_stage = LogisticRegression().fit(transform_cv, y_train)

    train_score = second_stage.score(transform_cv, y_train)

    test_score =second_stage.score(first_stage.transform(X_test), y_test)
    if train_score > test_score and naive_train_score > naive_test_score and test_score > naive_test_score and test_score > max(scores):
        print(i)
    #print(train_score > test_score)
    #print(naive_train_score > naive_test_score)
    #print(test_score > naive_test_score)
    #print(test_score > max(scores))